### Lab Machine Learning, del 2: Film rekommendations system
### Content Filtering Approach
- See file Lab_2_movie.ipynb for EDA and some initial experimenting.
#### Daniel Claesson, ITHS, AI-23
[länk till dataset på Kaggle](https://grouplens.org/datasets/movielens/)
Det använda datasetet heter "ml-latest".<br>
Målet med uppgiften är att ge användaren rekommendationer på filmer som kan passa en användare.

In [174]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [175]:
ratings = pd.read_csv('../data/ratings.csv')
movies = pd.read_csv('../data/movies.csv')
tags = pd.read_csv('../data/tags.csv')

#### Content Filtering Approach
- Detta är en content filtering approach till film rekommendation system. Innebär att baserat på en film som man anger, ska man få förslag på liknande filmer.
- Tidigare försök och EDA har gett erfarenheter som jag ska försöka implementera.
- Approachen att använda 'genre' och 'tag' för att särskilja filmerna verkar vara rätt.
- Men min tidigare approach gav genrellt sett för obalancerade kluster med K-means. Det var alltid ett kluster som var väldigt stort och många väldigt små kluster.
- Behov av att enkelt kunna variera hur många 'tags' och 'genres' som går in i analysen. Det finns över 50 tusen unika tags. Min första bedömning var att det skulle vara bra att använda många av dem, men det verkar inte ge bra resultat (ger fåtal stora kluster och väldigt många små kluster). Att reducera antalet 'tags' verkar vara bra för det syftet.
- I denna anlysen kommer jag ta bort dubbletter av samma uttryck i 'tags' och 'genre', kan troligen vara bra för analysen.
- Att använda en matris med filmer som rader och 'tags' och 'genre' som kolumner visade sig fungera i min första analys.
- Jag har skapat matrisen med pivot_table metoden i Pandas. Den fungerar bra, men när jag har filtrerat den input-dataframen på tags / genre, så blir input dataframen mindre, dvs jag gör en ofrivillig filtrering. Detta kommer jag lösa genom att göra filtreringen senare på själva output dataframen.
- Approach är att bygga min önskade dataframe så att i slutändan har kombinerat värdefull data från både 'genre' och 'tag', utan dubbletter, samt ges möjlighet att filtrera ut top x använda tags / genres.
- Kan tyvärr inte göra exakt så som jag tänkt från början. Datorns kapacitet räcker inte för att skapa en pivot table med alla 'tags' som index och alla 'movieId' som kolumner. Jag får reducera antalet 'tags', vilket gör att ganska många filmer som saknar tags tyvärr inte kommer med. Men det är fortfarande över 50 tusen filmer som är kvar.
- Min dator har 48GB minne, men den klarar precis köra igenom skriptet nedan....
##### Kort summering
##### Approach 1:
- Med information från om användartags, unika genres och kombinationer av genres skapar jag en matris med filmer som index och features som kolumner.
1. Information om 'tags': man väljer i koden hur många 'tags' man vill ha med. Jag har provat flera kombinationer och jag trodde fler skulle vara bättre. Men det visade sig att färre gav ett bättre resultat. Denna blir då de mest förekommande 'tags', bortsett från att dubbletter som återfinns i 
2. Information on unika genres, det är typ 20 unika genres.
3. information om kombinationer av genres, det är fler än 1600 sådana.
- En K-means klustrar datan. Tanken är att filmer i samma kluster ska likna varandra.
- De bästa filmerna från samma kluster, baserat på rating, rekommenderas till användaren.
- Under arbetets gång jobbade jag en del med att försöka få klustrens storlek att vara någorlunda lika stora, men det visade sig vara väldigt svårt. Ofta blev det några få stora kluster och sedan många små.
##### Approach 2:
- Den här approachen använder K-NN för att direkt i matrisen över filmer, hitta de som är närmast den angivna filmen, baserat på likhet på features.
- Den här metoden verkar ge mer tillförlitliga resultat.


### Skapar matrisen (filmer på rader, features på kolumner):

För att underlätta att ta bort dubbletter så omvandlas alla bokstäver till små.

In [176]:
movies['genres'] = movies['genres'].str.lower()
#movies['genres']

In [177]:
n_genres = 200 # välj de topp n valigaste genres som ska vara med
top_n_genres_list = movies['genres'].value_counts().head(n_genres).index.to_list()
#top_n_genres_list

In [178]:
movies_expl = movies.copy()
movies_expl['genres'] = movies_expl['genres'].str.split('|')
movies_exploded = movies_expl.explode('genres')
movies_exploded_df = movies_exploded['genres'].to_frame()

Tar fram alla unika 'genres'. Det är typ 20 st unika 'genres'.

In [179]:
unique_genres_array = movies_exploded_df['genres'].str.lower().unique()
unique_genres_df = pd.DataFrame(unique_genres_array)
unique_genres_df.columns = ['genre']
unique_genres_list = list(unique_genres_df['genre'].unique())
#unique_genres_list

Ange top_n_tags att använda i analysen.
- dubbletter ('tags' som redan finns som en 'genre') tas bort.

In [180]:
tags['tags'] = tags['tag'].str.lower() # gör om till små bokstäver

In [181]:
n_tags = 15 # ange top n tags att använda i analysen
top_n_tags_list = tags['tag'].value_counts().head(n_tags).index.to_list()

tags_list_final =[]
for item in top_n_tags_list:
    if item not in unique_genres_list:
        tags_list_final.append(item)

print(f'Det återstår {len(tags_list_final)} "tags" av {n_tags} valda "tags" efter att dubbletter tagits bort.')
#tags_list_final

Det återstår 11 "tags" av 15 valda "tags" efter att dubbletter tagits bort.


##### Skapar dataframes
- Börjar med tags:

In [182]:
n = 20000 # väljer hur många tags som ska med i analysen, ju fler man har här, ju mer minne krävs på datorn. Men ju färre man har, desto fler filmer blir bortsorterade.
tags_top_n = tags['tag'].value_counts().head(n)
tags_top_n_list = tags['tag'].value_counts().head(n).index.to_list()
tags_top_n

tag
sci-fi                    14319
atmospheric               12172
action                    10683
comedy                    10161
surreal                    9142
                          ...  
knee high boots              10
sexy leading actresses       10
Paz Vega                     10
religious conflict           10
UNLIKELY CRIMINALS           10
Name: count, Length: 20000, dtype: int64

In [183]:
tags_reduced = tags[tags['tag'].isin(tags_top_n_list)]
len(tags_reduced)

2045937

In [184]:
tags_pivot_table = pd.pivot_table(tags_reduced, index='movieId', columns='tag', aggfunc='size', fill_value=0)

In [185]:
tags_pivot_table

tag,"""damn dirty apes""","""found footage""",'60s feel,(s)vcd,*,*Good* Musicals,007,007 (series),01/10,01/11,...,zombie violence,zombies,zombification,zoo,zookeeper,zoologist,zoophilia,º,ºº,Álex de la Iglesia
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288849,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
tags_pivot_table = tags_pivot_table.loc[:,tags_list_final]
tags_pivot_table

tag,atmospheric,surreal,funny,visually appealing,twist ending,thought-provoking,dark comedy,based on a book,dystopia,cinematography,social commentary
movieId,,,,,,,,,,,
1,0,0,59,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,4,0,0,0
3,0,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
288753,0,0,0,0,0,0,0,0,0,0,0
288765,0,0,0,0,0,0,0,0,0,0,0
288849,0,0,0,0,0,0,0,0,0,0,0


In [187]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#scaler = StandardScaler()
scaler = MinMaxScaler()
scaled_tags_pivot_table = scaler.fit_transform(tags_pivot_table)
scaled_tags_pivot_table

array([[0.        , 0.        , 0.50862069, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00862069, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

Skapar pivot table för sammansatta genres, dvs de som sitter ihop med pipe '|':

In [188]:
movies_reduced = movies[movies['movieId'].isin(tags_pivot_table.index)]
movies

,movieId,title,genres
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy
1,2,Jumanji (1995),adventure|children|fantasy
2,3,Grumpier Old Men (1995),comedy|romance
3,4,Waiting to Exhale (1995),comedy|drama|romance
4,5,Father of the Bride Part II (1995),comedy
...,...,...,...
86532,288967,State of Siege: Temple Attack (2021),action|drama
86533,288971,Ouija Japan (2021),action|horror
86534,288975,The Men Who Made the Movies: Howard Hawks (1973),documentary
86535,288977,Skinford: Death Sentence (2023),crime|thriller


In [189]:
genres_pivot_table = pd.pivot_table(movies_reduced, index='movieId', columns='genres', aggfunc='size', fill_value=0)
genres_pivot_table

genres,(no genres listed),action,action|adventure,action|adventure|animation,action|adventure|animation|children,action|adventure|animation|children|comedy,action|adventure|animation|children|comedy|fantasy,action|adventure|animation|children|comedy|imax,action|adventure|animation|children|comedy|romance,action|adventure|animation|children|comedy|sci-fi,...,sci-fi|thriller|war,sci-fi|thriller|western,sci-fi|war,sci-fi|western,thriller,thriller|war,thriller|western,war,war|western,western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288765,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
288849,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Skapar pivot table för unika genres:

In [190]:
movies_genres_unique = movies[movies['movieId'].isin(genres_pivot_table.index)]
movies_genres_unique['genres'] = movies_genres_unique['genres'].str.split('|')
movies_exploded = movies_genres_unique.explode('genres')
#movies_exploded

C:\Users\danie\AppData\Local\Temp\ipykernel_9768\564276167.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_genres_unique['genres'] = movies_genres_unique['genres'].str.split('|')


In [191]:
genres_unique_pivot_table = pd.pivot_table(movies_exploded, index='movieId', columns='genres', aggfunc='size', fill_value=0)
genres_unique_pivot_table

genres,(no genres listed),action,adventure,animation,children,comedy,crime,documentary,drama,fantasy,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
movieId,,,,,,,,,,,,,,,,,,,,
1,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288753,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
288765,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0
288849,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Nu har jag tre pivot-tables. Alla har som index samma x antal filmer. Kolumnerna är i första dataframen "tags_pivot_table" top_n tags förutom dubbletter från unika genre.
- I den andra dataframen har som kolumner alla sammansatta kombinationer av 'genres' (action|drama|commedy etc).
- Den tredje har som kolumner alla unika 'genres' som 'drama', 'action' etc.
- Nästa steg är att sätta ihop dessa till en och samma dataframe. Eftersom de har samma index kan jag bara contatenera dem.

In [192]:
tags_genre = pd.concat([tags_pivot_table, genres_pivot_table], axis='columns')
tags_genre = pd.concat([tags_genre, genres_unique_pivot_table], axis='columns')

'tags_genre' är nu min dataframe som används både i Approach 1 och 2.

In [193]:
tags_genre

,atmospheric,surreal,funny,visually appealing,twist ending,thought-provoking,dark comedy,based on a book,dystopia,cinematography,...,film-noir,horror,imax,musical,mystery,romance,sci-fi,thriller,war,western
movieId,,,,,,,,,,,,,,,,,,,,,
1,0,0,59,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288753,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
288765,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
288849,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Klustring med K-means:

Nedan provar jag lite olika 'k', dvs antal kluster. Antalet kluster man vill använda kan nog variera lite, men man kan säkert göra 100 olika kluster. Men efter runt 250 kluster så sjunker inte inertia så mycket.

In [194]:
from sklearn.cluster import KMeans

#clusters = range(100,500,50)
#sum_squared_distance = [
#    KMeans(k, n_init="auto").fit(scaled_pivot_table).inertia_ for k in clusters
#]
#sum_squared_distance

#KMeans(200, n_init="auto").fit(scaled_pivot_table).inertia_

In [195]:
k = 250
kmeans = KMeans(k, n_init="auto").fit(tags_genre)

In [196]:
labels = pd.DataFrame(kmeans.labels_)
labels.value_counts().head(19)

0  
34     7968
60     4970
25     4344
134    2652
242    2297
144    2122
211    2040
129    1989
53     1945
91     1901
212    1739
172    1676
165    1511
155    1320
122    1311
140    1223
233    1078
208    1056
0      1029
Name: count, dtype: int64

In [197]:
labels

,0
0,65
1,181
2,53
3,184
4,60
...,...
51912,122
51913,122
51914,134
51915,184


In [198]:
movies_reduced['label']=labels #adderar klumnen 'labels', dvs resultat från K-means
#movies_reduced

C:\Users\danie\AppData\Local\Temp\ipykernel_9768\4235343258.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reduced['label']=labels #adderar klumnen 'labels', dvs resultat från K-means


In [199]:
movies_reduced[movies_reduced['movieId']==58559]

,movieId,title,genres,label
12223,58559,"Dark Knight, The (2008)",action|crime|drama|imax,144.0


Om jag tycker om filmen The Dark Knight så vill jag ha rekommenderat de bästa filmerna (låt oss säga filmerna med bäst rating, medel) i kluster med label som man kan se i outputen ovan.
- Beräknar medelbetyget för varje film och mergar betyget till samma dataframe (movies_reduced).

In [200]:
avg_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
#avg_ratings

In [201]:
movies_reduced = movies_reduced.merge(avg_ratings, left_on='movieId', right_on='movieId') # adderar medelbetyget per film till df

In [202]:
movies_reduced.head(5)

,movieId,title,genres,label,rating
0,1,Toy Story (1995),adventure|animation|children|comedy|fantasy,65.0,3.893508
1,2,Jumanji (1995),adventure|children|fantasy,181.0,3.278179
2,3,Grumpier Old Men (1995),comedy|romance,53.0,3.171271
3,4,Waiting to Exhale (1995),comedy|drama|romance,184.0,2.868395
4,5,Father of the Bride Part II (1995),comedy,60.0,3.076957


Baserat på movieId så får man rekommenderat top 10 filmer från samma kluster:
- exempelfilmer att prova:
- The Dark Knight (2008): 58559
- I'm not there (): 56286
- Batman Returns (1992): 1377
- Dirty Dancing: 1088
- Die Hard (1988): 1036
- The Matrix: 2571
- The Godfather (1972): 858
- Lord of the Rings: The Fellowship of the Ring, The (2001): 4993
- Silence of the Lambs, The (1991): 593

In [203]:
movie_Id = 2571# ange movieId för vilken du vill ha rekommendation av liknande filmer

### Approach 1: rekommenderar de bästa filmerna (baserat på rating), baserat på filmer i samma kluster.

In [204]:
movie_label = int(movies_reduced[movies_reduced['movieId'] == movie_Id]['label'])
movies_gb = movies_reduced.groupby('label')
movies_reduced.loc[movies_gb.groups[movie_label]].sort_values('rating',ascending=False).head(10)

C:\Users\danie\AppData\Local\Temp\ipykernel_9768\3754119158.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  movie_label = int(movies_reduced[movies_reduced['movieId'] == movie_Id]['label'])


,movieId,title,genres,label,rating
26291,137032,The Perfect Neighbor (2005),drama|thriller,212.0,5.000000
28097,143579,Violet's Visit (1997),comedy|romance,212.0,5.000000
23302,125920,Buffalo Girls (2012),action,212.0,4.500000
29849,150635,The Battle Of The Villa Fiorita (1965),drama,212.0,4.500000
36353,173127,"Hamlet, Prince of Denmark (1961)",drama|fantasy,212.0,4.500000
36436,173363,American Meth (2008),documentary,212.0,4.500000
11216,54326,"Sierra, La (2005)",documentary,212.0,4.333333
34485,167196,1. Mai (2008),drama,212.0,4.250000
27152,140431,The Biscuit Eater (1940),(no genres listed),212.0,4.250000
23016,124937,Willow and Wind (2000),(no genres listed),212.0,4.200000


### Approach 2: rekommenderar filmer baserat på närhet mha K-nn, direkt från film/features matrisen.

In [205]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10)
knn.fit(tags_genre)
movie_features = tags_genre.loc[movie_Id].values.reshape(1,-1) # hämtar features från dataframen "tags_genre" för filmen man vill kolla
distans, ind = knn.kneighbors(movie_features)
similar_movies_indices = ind[0][1:]
similar_movies = tags_genre.index[similar_movies_indices]
list_of_movieIds = similar_movies.to_list()

list_of_similar_movies = []
print(f"\nFilmrekommendation:\nEftersom du gillar {movies_reduced.loc[movies_reduced['movieId'] == movie_Id, 'title'].iloc[0]},\nså rekommenderas dessa liknande filmer: \n")
for item in list_of_movieIds:
    print(f"{movies_reduced.loc[movies_reduced['movieId'] == item, 'title'].iloc[0]}")


c:\Users\danie\.virtualenvs\Machine_Learning_Daniel_Claesson-bhjbabyT\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(



Filmrekommendation:
Eftersom du gillar Matrix, The (1999),
så rekommenderas dessa liknande filmer: 

V for Vendetta (2006)
Gattaca (1997)
Brazil (1985)
Children of Men (2006)
Dark City (1998)
Her (2013)
Moon (2009)
Donnie Darko (2001)
Eternal Sunshine of the Spotless Mind (2004)
